In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [2]:
%cd '/content/drive/MyDrive'

dataset = 'hand_gesture_detection/data.csv'
model_save_path = 'hand_gesture_detection/model/keypoint_classifier.hdf5'
tflite_save_path = 'hand_gesture_detection/model/keypoint_classifier.tflite'

/content/drive/MyDrive


In [3]:
NUM_CLASSES = 2

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75)

In [7]:
np.array(X_test[0].shape)

array([42])

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 1092 (4.27 KB)
Trainable params: 1092 (4.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [11]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
24/24 [==============================] - ETA: 0s - loss: 0.4313 - accuracy: 0.8080
Epoch 1: saving model to hand_gesture_detection/model/keypoint_classifier.hdf5
24/24 [==============================] - 0s 16ms/step - loss: 0.4313 - accuracy: 0.8080 - val_loss: 0.2085 - val_accuracy: 1.0000
Epoch 2/1000
24/24 [==============================] - ETA: 0s - loss: 0.4521 - accuracy: 0.8080
Epoch 2: saving model to hand_gesture_detection/model/keypoint_classifier.hdf5
24/24 [==============================] - 0s 6ms/step - loss: 0.4521 - accuracy: 0.8080 - val_loss: 0.2740 - val_accuracy: 1.0000
Epoch 3/1000
 1/24 [>.............................] - ETA: 0s - loss: 0.5085 - accuracy: 0.6875
Epoch 3: saving model to hand_gesture_detection/model/keypoint_classifier.hdf5
24/24 [==============================] - 0s 5ms/step - loss: 0.4611 - accuracy: 0.8000 - val_loss: 0.2331 - val_accuracy: 1.0000
Epoch 4/1000
22/24 [==========================>...] - ETA: 0s - loss: 0.4035 - accuracy

In [14]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=16)

8/8 [==============================] - 0s 3ms/step - loss: 0.1540 - accuracy: 1.0000


In [15]:
model = tf.keras.models.load_model(model_save_path)

In [16]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 93ms/step
[0.06422728 0.9357727 ]
1
